In [1]:
from pylablib.legacy.aux_libs.devices import Thorlabs
import visa
import serial
import time
import numpy as np
import plotly.graph_objects as go
from scipy.interpolate import interp1d

rm = visa.ResourceManager()
print(rm.list_resources())
sourcemeter = rm.open_resource('USB0::0x1313::0x8078::P0014759::INSTR', read_termination='\n')

ser = serial.Serial('COM4')

def readRot():
    ser.flushInput()
    ser.flushOutput()
    ser.write(b'0gp')
    r = ser.readline()
    return int((r.rstrip()).decode("utf-8")[3:], 16)

def writeRot(val):
    ser.flushInput()
    ser.flushOutput()
    w = '0ma'+format(val, 'X').zfill(8)
    ser.write(w.encode("utf-8"))

#ser.close()

C:\Users\TeraLab\anaconda3\lib\site-packages\pylablib\legacy\__init__.py:2: FutureWarning: legacy (pre 1.0) code will be removed in future versions
  warnings.warn("legacy (pre 1.0) code will be removed in future versions",FutureWarning)


('USB0::0x05E6::0x6500::04479852::INSTR', 'USB0::0x1313::0x8078::P0014759::INSTR', 'ASRL1::INSTR', 'ASRL4::INSTR', 'ASRL5::INSTR', 'ASRL6::INSTR', 'GPIB0::11::INSTR', 'GPIB0::15::INSTR', 'GPIB0::16::INSTR', 'GPIB0::23::INSTR', 'GPIB0::24::INSTR')


# Actual Position of Rotator

In [3]:
writeRot(14000)
readRot()

13998

# Measure Calibration Curve

In [4]:
rot_from = 14000
rot_to = 27000
step = 50
calib_file = 'probePowerCalibration_new.dat'

writeRot(rot_from)
time.sleep(5)

def one_step(pos):
    writeRot(pos)
    time.sleep(1)
    print(pos)
    return float(sourcemeter.query('measure:power?'))

measurement = [[i, one_step(i)] for i in range(rot_from, rot_to, step)]
np.savetxt(calib_file, measurement)

14000
14050
14100
14150
14200
14250
14300
14350
14400
14450
14500
14550
14600
14650
14700
14750
14800
14850
14900
14950
15000
15050
15100
15150
15200
15250
15300
15350
15400
15450
15500
15550
15600
15650
15700
15750
15800
15850
15900
15950
16000
16050
16100
16150
16200
16250
16300
16350
16400
16450
16500
16550
16600
16650
16700
16750
16800
16850
16900
16950
17000
17050
17100
17150
17200
17250
17300
17350
17400
17450
17500
17550
17600
17650
17700
17750
17800
17850
17900
17950
18000
18050
18100
18150
18200
18250
18300
18350
18400
18450
18500
18550
18600
18650
18700
18750
18800
18850
18900
18950
19000
19050
19100
19150
19200
19250
19300
19350
19400
19450
19500
19550
19600
19650
19700
19750
19800
19850
19900
19950
20000
20050
20100
20150
20200
20250
20300
20350
20400
20450
20500
20550
20600
20650
20700
20750
20800
20850
20900
20950
21000
21050
21100
21150
21200
21250
21300
21350
21400
21450
21500
21550
21600
21650
21700
21750
21800
21850
21900
21950
22000
22050
22100
22150
22200
22250
2230

# Load Calibration

In [9]:
from scipy.optimize import curve_fit

calib_File = 'probePowerCalibration_new.dat'
calib_data = np.loadtxt(calib_File)

def MalusLaw(x, A, T, x0, off):
    return A*np.cos(2*np.pi*(x-x0)/T+np.pi/2)**2+off

def getIntensityPos(y, A, T, x0, off):
    return (T*np.arcsin(np.sqrt((y-off)/A)))/(2*np.pi)+x0

popt, pcov = curve_fit(MalusLaw, calib_data[:,0], calib_data[:,1], p0= [ np.max(calib_data[:,1]),  2*(calib_data[-1,0]-calib_data[0,0]), (calib_data[-1,0]+calib_data[0,0])/2,  0]) 
print(popt)

fig = go.Figure()

fig.add_trace(go.Scatter(x=calib_data[:,0], y=calib_data[:,1], mode='lines'))
fig.add_trace(go.Scatter(x=calib_data[:,0], y=MalusLaw(calib_data[:,0], *popt), mode='lines+markers'))

fig.show()

def setPower(power):
    power = power/1000 # z mW -> W 
    pos = getIntensityPos(power,*popt)
    #pos = find_power(power, fnc, 23000,69000)
    writeRot(int(pos))
    time.sleep(0.1)

[1.15921253e-01 6.86203758e+04 1.39393862e+04 6.09224384e-05]


# Consistency Check:

In [117]:
MalusLaw(getIntensityPos(0.05,*popt), *popt)

0.049999999999999975

# Try to set Power:

In [10]:
setPower(5)

# Close serial port:

In [2]:
ser.close()

NameError: name 'ser' is not defined